In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

import requests
import json
import geopandas as gpd

# WeGo Public Transit

[WeGo Public Transit](https://www.wegotransit.com/) is a public transit system serving the Greater Nashville
and Davidson County area. WeGo provides local and regional bus routes, the WeGo Star train service connecting
Lebanon to downtown Nashville, along with several other transit services.

In this project, your goal will be to analyze data from WeGo's bus routes and attempt to identify intersections
and other locations which experience frequent delays. This analysis can look by route, by direction of travel,
and by day or day of the week.

You can also consider time of day in your investigation (for example, comparing
the AM and PM rush on Monday through Friday).

You can obtain the data for this project through Swiftly. You will be provided with an API key to  access this data.
The API that you will be using is the [Speed Map API](https://swiftly-inc.stoplight.io/docs/standalone/573bc3fb95867-speed-map),
which provides average speed information for a single route and direction over a given date range.

Note that the API can return either speed data at the stop segment level or return high resolution data,
which provides information for stop segments of approximately 25 meters along with more detailed speed and duration information.

Please take the time to carefully read the API documentation so that you are familiar with the query parameter option
and understand the response data.

You can get more information about these routes, such as route numbers and stop information,
using the [Agency Routes API](https://swiftly-inc.stoplight.io/docs/standalone/5c0bd000bb0f2-agency-routes).
A user-friendly map showing all routes is available at https://www.wegotransit.com/ride/maps-schedules/bus/. 

If time allows, investigate all routes, but start with the high ridership and frequent routes:
* 3: West End
* 7: Hillsboro
* 22: Bordeaux
* 23: Dickerson Pike
* 50: Charlotte Pike
* 52: Nolensville Pike
* 55: Murfreesboro Pike
* 56: Gallatin Pike

Bottlenecks - constant/irregular
(constant)
*ignoring bus stop sections
Finding downtown vs suburbs bottlenecks
1st Quartile of each route for constant
*Delayed data

Focus on Inbound/Outbound
(inbound)

Time categories -
(6am - 9am AM Peak)
9am-230pm Off Peak
230pm-6pm PM Peak
6pm-6am Overnight
Day of Interest
Saturday
Sunday
Holidays*
(M/F)
(Tues-Thurs)
(5/25-7/26)
(1/25-5/24)


# API access
Agency Info: agency_details, routes, schedule

Speed-map: Check all, speed-map



In [2]:
with open('../data/WeGo.json') as fi:
    credentials = json.load(fi)

In [3]:
Authorization = credentials['Authorization']

In [4]:
# URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

# params_geo3 = {
#     'direction': '1',
#     'startDate': '01-07-2022',
#     'endDate': '01-26-2022',
#     'daysOfWeek': '1,2,3,4,5',
#     'beginTime': '06:00',
#     'endTime': '09:00',
#     'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
#     'format': 'geojson',
#     'resolution': 'hiRes'
# }

# headers = {'Authorization': Authorization,
#           'Content-Type': 'application/json'}

# response_geo3 = requests.get(URL, params = params_geo3, headers = headers)
# response_geo3

# # dig into json to get list of line-segment dicts
# segments3 = response_geo3.json()['features']

# # convert to GeoDataFrame the quick way
# fullweekgeo = gpd.GeoDataFrame.from_features(segments3)

# fullweekgeo

In [34]:
def concat_frames_columns(geo_response, response, date_range, time_range='6am-9am', days='m-f', in_or_out='inbound'):
    """This function pulls in two API requests, puts one in a geodataframe the other in a dataframe and concats desired columns"""
    geo_df = gpd.GeoDataFrame.from_features(geo_response.json()['features'])
    res_df = pd.DataFrame(response.json()['data']['segments'])
    full_df = pd.concat([geo_df, res_df[['countForSegment', 'percent20Secs']]], axis = 1)
    full_df['dates'] = date_range
    full_df['times'] = time_range
    full_df['days_of_week'] = days
    full_df['direction'] = in_or_out
    full_df['max_seg_count'] = full_df['countForSegment'].max()
    full_df['proportion_of_segments'] = full_df['countForSegment']/full_df['max_seg_count']
    
    return full_df

# Start of M-F inbound df, sorted by 30 day chunks Jan 25-July26

In [35]:
# import pygeos
# print(f"pygeos version:  {pygeos.__version__}")
# print(f"geos version: {pygeos.geos_version}")
# print(f"use pygeos? {gpd.options.use_pygeos}")

In [36]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '1',
    'startDate': '01-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '02-24-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '1',
    'startDate': '01-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '02-24-2022',
    'endTime' : '09:00',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

mfam_jantofeb = concat_frames_columns(requests.get(URL, params = params, headers = headers),
                                     requests.get(URL, params = params1, headers = headers), "1/25/22-2/24/22")

#mfam_jantofeb = mfam_jantofeb.loc[mfam_jantofeb['countForSegment'] > mfam_jantofeb['countForSegment'].max()/2]

mfam_jantofeb

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,times,days_of_week,direction,max_seg_count,proportion_of_segments
0,"LINESTRING (-86.83200 36.15252, -86.83173 36.1...",CHA39AEF,Charlotte Ave & 39th Ave N Eb,CXISYLHE,37th Ave/Sylvan Heights Station Inbound,24.375,2.139055,11.395219,7.813,3.546,...,1.431,1.285,253,1.490,1/25/22-2/24/22,6am-9am,m-f,inbound,262,0.965649
1,"LINESTRING (-86.86416 36.14888, -86.86390 36.1...",CHABALEF,Charlotte Ave & Balmy Ave Eb,CHALELEM,Charlotte Pike & Lellyett St Eb,23.507,1.545192,15.212996,4.745,1.828,...,1.247,1.064,245,1.312,1/25/22-2/24/22,6am-9am,m-f,inbound,262,0.935115
2,"LINESTRING (-86.79194 36.16148, -86.79169 36.1...",CXI17AVE,17th Ave Station Inbound,CXINGULC,11th Ave / N Gulch Station Inbound,24.691,2.323759,10.625456,5.661,3.087,...,1.800,1.280,253,1.921,1/25/22-2/24/22,6am-9am,m-f,inbound,262,0.965649
3,"LINESTRING (-86.84193 36.15216, -86.84166 36.1...",CXIRICHL,46th Ave/Richland Park Station Inbound,CXISYLPA,42nd Ave/Sylvan Park Station Inbound,24.527,2.678840,9.155829,15.590,4.099,...,1.683,0.969,256,1.843,1/25/22-2/24/22,6am-9am,m-f,inbound,262,0.977099
4,"LINESTRING (-86.87277 36.14445, -86.87258 36.1...",PREWILNN,Premier Dr & Wilclay Dr Nb,CXIAMERI,American Station Inbound,24.433,4.294515,5.689350,27.961,7.016,...,2.407,1.326,241,2.770,1/25/22-2/24/22,6am-9am,m-f,inbound,262,0.919847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,"LINESTRING (-86.87143 36.14546, -86.87124 36.1...",CXIAMERI,American Station Inbound,CHABALEF,Charlotte Ave & Balmy Ave Eb,24.662,2.607100,9.459553,7.082,3.703,...,1.758,1.368,250,1.928,1/25/22-2/24/22,6am-9am,m-f,inbound,262,0.954198
563,"LINESTRING (-86.80505 36.15676, -86.80479 36.1...",CXICENTE,23rd Ave / Centennial Station Inbound,CHA20AEF,Charlotte Ave & 20th Ave Eb,24.161,1.907988,12.663078,6.088,2.620,...,1.448,1.195,258,1.531,1/25/22-2/24/22,6am-9am,m-f,inbound,262,0.984733
564,"LINESTRING (-86.85058 36.15184, -86.85031 36.1...",CXI53AVE,53rd Ave Station Inbound,CXIRICHL,46th Ave/Richland Park Station Inbound,24.270,7.806658,3.108885,128.807,19.048,...,1.614,1.249,260,1.801,1/25/22-2/24/22,6am-9am,m-f,inbound,262,0.992366
565,"LINESTRING (-86.83597 36.15238, -86.83570 36.1...",CXISYLPA,42nd Ave/Sylvan Park Station Inbound,CHA39AEF,Charlotte Ave & 39th Ave N Eb,23.766,3.668957,6.477590,10.161,5.796,...,2.003,0.990,256,2.325,1/25/22-2/24/22,6am-9am,m-f,inbound,262,0.977099


In [37]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '1',
    'startDate': '02-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '03-26-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '1',
    'startDate': '02-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '03-26-2022',
    'endTime' : '09:00',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

mfam_febtomar = concat_frames_columns(requests.get(URL, params = params, headers = headers),
                                     requests.get(URL, params = params1, headers = headers), "2/25/22-3/26/22")

#mfam_febtomar = mfam_febtomar.loc[mfam_febtomar['countForSegment'] > mfam_febtomar['countForSegment'].max()/2]

mfam_febtomar

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,times,days_of_week,direction,max_seg_count,proportion_of_segments
0,"LINESTRING (-86.83200 36.15252, -86.83173 36.1...",CHA39AEF,Charlotte Ave & 39th Ave N Eb,CXISYLHE,37th Ave/Sylvan Heights Station Inbound,24.375,2.434798,10.011097,15.501,5.232,...,1.445,1.212,188,1.495,2/25/22-3/26/22,6am-9am,m-f,inbound,190,0.989474
1,"LINESTRING (-86.86416 36.14888, -86.86390 36.1...",CHABALEF,Charlotte Ave & Balmy Ave Eb,CHALELEM,Charlotte Pike & Lellyett St Eb,23.507,1.522770,15.437000,7.156,1.712,...,1.215,1.006,183,1.311,2/25/22-3/26/22,6am-9am,m-f,inbound,190,0.963158
2,"LINESTRING (-86.79194 36.16148, -86.79169 36.1...",CXI17AVE,17th Ave Station Inbound,CXINGULC,11th Ave / N Gulch Station Inbound,24.691,2.309017,10.693295,6.581,3.052,...,1.731,1.446,179,1.823,2/25/22-3/26/22,6am-9am,m-f,inbound,190,0.942105
3,"LINESTRING (-86.84193 36.15216, -86.84166 36.1...",CXIRICHL,46th Ave/Richland Park Station Inbound,CXISYLPA,42nd Ave/Sylvan Park Station Inbound,24.527,2.724216,9.003324,8.830,4.169,...,1.802,1.350,190,1.932,2/25/22-3/26/22,6am-9am,m-f,inbound,190,1.000000
4,"LINESTRING (-86.87277 36.14445, -86.87258 36.1...",PREWILNN,Premier Dr & Wilclay Dr Nb,CXIAMERI,American Station Inbound,24.433,4.248310,5.751228,22.778,6.441,...,2.617,1.577,187,3.019,2/25/22-3/26/22,6am-9am,m-f,inbound,190,0.984211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,"LINESTRING (-86.87143 36.14546, -86.87124 36.1...",CXIAMERI,American Station Inbound,CHABALEF,Charlotte Ave & Balmy Ave Eb,24.662,2.694247,9.153578,10.627,3.869,...,1.780,0.921,186,1.992,2/25/22-3/26/22,6am-9am,m-f,inbound,190,0.978947
563,"LINESTRING (-86.80505 36.15676, -86.80479 36.1...",CXICENTE,23rd Ave / Centennial Station Inbound,CHA20AEF,Charlotte Ave & 20th Ave Eb,24.161,1.991113,12.134419,6.393,2.735,...,1.452,1.215,186,1.556,2/25/22-3/26/22,6am-9am,m-f,inbound,190,0.978947
564,"LINESTRING (-86.85058 36.15184, -86.85031 36.1...",CXI53AVE,53rd Ave Station Inbound,CXIRICHL,46th Ave/Richland Park Station Inbound,24.270,7.436254,3.263740,50.733,22.845,...,1.637,1.227,189,1.771,2/25/22-3/26/22,6am-9am,m-f,inbound,190,0.994737
565,"LINESTRING (-86.83597 36.15238, -86.83570 36.1...",CXISYLPA,42nd Ave/Sylvan Park Station Inbound,CHA39AEF,Charlotte Ave & 39th Ave N Eb,23.766,3.872550,6.137041,10.443,6.006,...,2.216,1.612,189,2.558,2/25/22-3/26/22,6am-9am,m-f,inbound,190,0.994737


In [38]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '1',
    'startDate': '03-27-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '04-26-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '1',
    'startDate': '03-27-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '04-26-2022',
    'endTime' : '09:00',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

mfam_martoapr = concat_frames_columns(requests.get(URL, params = params, headers = headers),
                                     requests.get(URL, params = params1, headers = headers), "3/27/22-4/26/22")

#mfam_martoapr = mfam_martoapr.loc[mfam_martoapr['countForSegment'] > mfam_martoapr['countForSegment'].max()/2]

mfam_martoapr

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,times,days_of_week,direction,max_seg_count,proportion_of_segments
0,"LINESTRING (-86.83200 36.15252, -86.83173 36.1...",CHA39AEF,Charlotte Ave & 39th Ave N Eb,CXISYLHE,37th Ave/Sylvan Heights Station Inbound,24.375,2.668957,9.132781,8.657,5.722,...,1.447,0.879,234,1.495,3/27/22-4/26/22,6am-9am,m-f,inbound,242,0.966942
1,"LINESTRING (-86.86416 36.14888, -86.86390 36.1...",CHABALEF,Charlotte Ave & Balmy Ave Eb,CHALELEM,Charlotte Pike & Lellyett St Eb,23.507,1.519463,15.470597,5.540,1.733,...,1.257,0.910,229,1.330,3/27/22-4/26/22,6am-9am,m-f,inbound,242,0.946281
2,"LINESTRING (-86.79194 36.16148, -86.79169 36.1...",CXI17AVE,17th Ave Station Inbound,CXINGULC,11th Ave / N Gulch Station Inbound,24.691,2.374926,10.396534,4.833,3.090,...,1.799,1.484,231,1.949,3/27/22-4/26/22,6am-9am,m-f,inbound,242,0.954545
3,"LINESTRING (-86.84193 36.15216, -86.84166 36.1...",CXIRICHL,46th Ave/Richland Park Station Inbound,CXISYLPA,42nd Ave/Sylvan Park Station Inbound,24.527,2.656868,9.231546,20.947,3.909,...,1.760,1.401,234,1.881,3/27/22-4/26/22,6am-9am,m-f,inbound,242,0.966942
4,"LINESTRING (-86.87277 36.14445, -86.87258 36.1...",PREWILNN,Premier Dr & Wilclay Dr Nb,CXIAMERI,American Station Inbound,24.433,4.768948,5.123352,40.488,7.445,...,2.621,1.933,233,3.042,3/27/22-4/26/22,6am-9am,m-f,inbound,242,0.962810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,"LINESTRING (-86.87143 36.14546, -86.87124 36.1...",CXIAMERI,American Station Inbound,CHABALEF,Charlotte Ave & Balmy Ave Eb,24.662,2.800172,8.807316,8.896,4.011,...,1.870,1.555,233,2.048,3/27/22-4/26/22,6am-9am,m-f,inbound,242,0.962810
563,"LINESTRING (-86.80505 36.15676, -86.80479 36.1...",CXICENTE,23rd Ave / Centennial Station Inbound,CHA20AEF,Charlotte Ave & 20th Ave Eb,24.161,1.985046,12.171506,7.662,2.684,...,1.478,1.269,240,1.578,3/27/22-4/26/22,6am-9am,m-f,inbound,242,0.991736
564,"LINESTRING (-86.85058 36.15184, -86.85031 36.1...",CXI53AVE,53rd Ave Station Inbound,CXIRICHL,46th Ave/Richland Park Station Inbound,24.270,7.862739,3.086711,87.073,20.429,...,1.638,1.151,234,1.727,3/27/22-4/26/22,6am-9am,m-f,inbound,242,0.966942
565,"LINESTRING (-86.83597 36.15238, -86.83570 36.1...",CXISYLPA,42nd Ave/Sylvan Park Station Inbound,CHA39AEF,Charlotte Ave & 39th Ave N Eb,23.766,4.034282,5.891011,12.483,6.295,...,2.256,1.695,234,2.542,3/27/22-4/26/22,6am-9am,m-f,inbound,242,0.966942


In [39]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '1',
    'startDate': '04-27-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '05-25-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '1',
    'startDate': '04-27-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '05-25-2022',
    'endTime' : '09:00',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

mfam_aprtomay = concat_frames_columns(requests.get(URL, params = params, headers = headers),
                                     requests.get(URL, params = params1, headers = headers), "4/27/22-5/25/22")

#mfam_aprtomay = mfam_aprtomay.loc[mfam_aprtomay['countForSegment'] > mfam_aprtomay['countForSegment'].max()/2]

mfam_aprtomay

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,times,days_of_week,direction,max_seg_count,proportion_of_segments
0,"LINESTRING (-86.83200 36.15252, -86.83173 36.1...",CHA39AEF,Charlotte Ave & 39th Ave N Eb,CXISYLHE,37th Ave/Sylvan Heights Station Inbound,24.375,2.875554,8.476627,32.762,5.496,...,1.462,1.254,242,1.525,4/27/22-5/25/22,6am-9am,m-f,inbound,242,1.000000
1,"LINESTRING (-86.86416 36.14888, -86.86390 36.1...",CHABALEF,Charlotte Ave & Balmy Ave Eb,CHALELEM,Charlotte Pike & Lellyett St Eb,23.507,1.516862,15.497125,5.337,1.714,...,1.235,0.969,232,1.312,4/27/22-5/25/22,6am-9am,m-f,inbound,242,0.958678
2,"LINESTRING (-86.79194 36.16148, -86.79169 36.1...",CXI17AVE,17th Ave Station Inbound,CXINGULC,11th Ave / N Gulch Station Inbound,24.691,2.418363,10.209799,5.667,3.301,...,1.857,1.348,226,1.935,4/27/22-5/25/22,6am-9am,m-f,inbound,242,0.933884
3,"LINESTRING (-86.84193 36.15216, -86.84166 36.1...",CXIRICHL,46th Ave/Richland Park Station Inbound,CXISYLPA,42nd Ave/Sylvan Park Station Inbound,24.527,2.684471,9.136623,6.998,4.191,...,1.756,1.429,242,1.865,4/27/22-5/25/22,6am-9am,m-f,inbound,242,1.000000
4,"LINESTRING (-86.87277 36.14445, -86.87258 36.1...",PREWILNN,Premier Dr & Wilclay Dr Nb,CXIAMERI,American Station Inbound,24.433,4.265880,5.727540,10.482,6.960,...,2.493,1.779,234,2.980,4/27/22-5/25/22,6am-9am,m-f,inbound,242,0.966942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,"LINESTRING (-86.87143 36.14546, -86.87124 36.1...",CXIAMERI,American Station Inbound,CHABALEF,Charlotte Ave & Balmy Ave Eb,24.662,2.581885,9.551936,6.928,3.681,...,1.780,1.181,235,1.931,4/27/22-5/25/22,6am-9am,m-f,inbound,242,0.971074
562,"LINESTRING (-86.80505 36.15676, -86.80479 36.1...",CXICENTE,23rd Ave / Centennial Station Inbound,CHA20AEF,Charlotte Ave & 20th Ave Eb,24.161,1.968857,12.271587,10.811,2.569,...,1.467,1.149,238,1.557,4/27/22-5/25/22,6am-9am,m-f,inbound,242,0.983471
563,"LINESTRING (-86.85058 36.15184, -86.85031 36.1...",CXI53AVE,53rd Ave Station Inbound,CXIRICHL,46th Ave/Richland Park Station Inbound,24.270,6.782868,3.578132,84.840,18.202,...,1.621,1.184,242,1.708,4/27/22-5/25/22,6am-9am,m-f,inbound,242,1.000000
564,"LINESTRING (-86.88379 36.14136, -86.88404 36.1...",ANNCHAWM,Annex Ave & Charlotte Pike Wb,PREWILNN,Premier Dr & Wilclay Dr Nb,24.837,3.491268,7.114034,8.314,4.714,...,2.457,1.605,239,2.659,4/27/22-5/25/22,6am-9am,m-f,inbound,242,0.987603


In [40]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '1',
    'startDate': '05-26-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '06-25-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '1',
    'startDate': '05-26-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '06-25-2022',
    'endTime' : '09:00',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

mfam_maytojun = concat_frames_columns(requests.get(URL, params = params, headers = headers),
                                     requests.get(URL, params = params1, headers = headers), "4/27/22-5/25/22")

#mfam_maytojun = mfam_maytojun.loc[mfam_maytojun['countForSegment'] > mfam_maytojun['countForSegment'].max()/2]

mfam_maytojun

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,times,days_of_week,direction,max_seg_count,proportion_of_segments
0,"LINESTRING (-86.83200 36.15252, -86.83173 36.1...",CHA39AEF,Charlotte Ave & 39th Ave N Eb,CXISYLHE,37th Ave/Sylvan Heights Station Inbound,24.375,2.532788,9.623782,12.290,5.203,...,1.429,1.139,245,1.498,4/27/22-5/25/22,6am-9am,m-f,inbound,252,0.972222
1,"LINESTRING (-86.86416 36.14888, -86.86390 36.1...",CHABALEF,Charlotte Ave & Balmy Ave Eb,CHALELEM,Charlotte Pike & Lellyett St Eb,23.507,1.512034,15.546608,5.910,1.685,...,1.243,1.007,235,1.331,4/27/22-5/25/22,6am-9am,m-f,inbound,252,0.932540
2,"LINESTRING (-86.79194 36.16148, -86.79169 36.1...",CXI17AVE,17th Ave Station Inbound,CXINGULC,11th Ave / N Gulch Station Inbound,24.691,2.416913,10.215924,11.560,3.342,...,1.761,1.389,240,1.884,4/27/22-5/25/22,6am-9am,m-f,inbound,252,0.952381
3,"LINESTRING (-86.84193 36.15216, -86.84166 36.1...",CXIRICHL,46th Ave/Richland Park Station Inbound,CXISYLPA,42nd Ave/Sylvan Park Station Inbound,24.527,2.760904,8.883684,15.635,4.320,...,1.712,1.401,249,1.894,4/27/22-5/25/22,6am-9am,m-f,inbound,252,0.988095
4,"LINESTRING (-86.87277 36.14445, -86.87258 36.1...",PREWILNN,Premier Dr & Wilclay Dr Nb,CXIAMERI,American Station Inbound,24.433,4.439903,5.503048,10.256,7.044,...,2.709,1.747,247,3.081,4/27/22-5/25/22,6am-9am,m-f,inbound,252,0.980159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,"LINESTRING (-86.87143 36.14546, -86.87124 36.1...",CXIAMERI,American Station Inbound,CHABALEF,Charlotte Ave & Balmy Ave Eb,24.662,2.638152,9.348210,8.215,3.704,...,1.760,0.971,244,1.990,4/27/22-5/25/22,6am-9am,m-f,inbound,252,0.968254
562,"LINESTRING (-86.80505 36.15676, -86.80479 36.1...",CXICENTE,23rd Ave / Centennial Station Inbound,CHA20AEF,Charlotte Ave & 20th Ave Eb,24.161,2.047355,11.801080,19.906,2.980,...,1.487,0.892,248,1.562,4/27/22-5/25/22,6am-9am,m-f,inbound,252,0.984127
563,"LINESTRING (-86.85058 36.15184, -86.85031 36.1...",CXI53AVE,53rd Ave Station Inbound,CXIRICHL,46th Ave/Richland Park Station Inbound,24.270,7.632524,3.179813,98.684,19.024,...,1.607,1.337,250,1.763,4/27/22-5/25/22,6am-9am,m-f,inbound,252,0.992063
564,"LINESTRING (-86.83597 36.15238, -86.83570 36.1...",CXISYLPA,42nd Ave/Sylvan Park Station Inbound,CHA39AEF,Charlotte Ave & 39th Ave N Eb,23.766,3.779992,6.287315,11.217,6.497,...,2.013,1.427,249,2.253,4/27/22-5/25/22,6am-9am,m-f,inbound,252,0.988095


In [41]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '1',
    'startDate': '06-26-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '07-26-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '1',
    'startDate': '06-26-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '07-26-2022',
    'endTime' : '09:00',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

mfam_juntojul = concat_frames_columns(requests.get(URL, params = params, headers = headers),
                                     requests.get(URL, params = params1, headers = headers), "4/27/22-5/25/22")

#mfam_juntojul = mfam_juntojul.loc[mfam_juntojul['countForSegment'] > mfam_juntojul['countForSegment'].max()/2]

mfam_juntojul

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,times,days_of_week,direction,max_seg_count,proportion_of_segments
0,"LINESTRING (-86.83200 36.15252, -86.83173 36.1...",CHA39AEF,Charlotte Ave & 39th Ave N Eb,CXISYLHE,37th Ave/Sylvan Heights Station Inbound,24.375,2.659086,9.166684,9.689,6.150,...,1.445,1.279,233,1.505,4/27/22-5/25/22,6am-9am,m-f,inbound,240,0.970833
1,"LINESTRING (-86.86416 36.14888, -86.86390 36.1...",CHABALEF,Charlotte Ave & Balmy Ave Eb,CHALELEM,Charlotte Pike & Lellyett St Eb,23.507,1.532767,15.336317,5.864,1.715,...,1.252,0.973,227,1.325,4/27/22-5/25/22,6am-9am,m-f,inbound,240,0.945833
2,"LINESTRING (-86.79194 36.16148, -86.79169 36.1...",CXI17AVE,17th Ave Station Inbound,CXINGULC,11th Ave / N Gulch Station Inbound,24.691,2.286261,10.799729,16.466,2.987,...,1.685,0.747,226,1.815,4/27/22-5/25/22,6am-9am,m-f,inbound,240,0.941667
3,"LINESTRING (-86.84193 36.15216, -86.84166 36.1...",CXIRICHL,46th Ave/Richland Park Station Inbound,CXISYLPA,42nd Ave/Sylvan Park Station Inbound,24.527,2.632811,9.315898,14.720,4.046,...,1.678,1.031,233,1.858,4/27/22-5/25/22,6am-9am,m-f,inbound,240,0.970833
4,"LINESTRING (-86.87277 36.14445, -86.87258 36.1...",PREWILNN,Premier Dr & Wilclay Dr Nb,CXIAMERI,American Station Inbound,24.433,4.575500,5.339963,23.075,7.523,...,2.598,1.138,234,3.154,4/27/22-5/25/22,6am-9am,m-f,inbound,240,0.975000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,"LINESTRING (-86.87143 36.14546, -86.87124 36.1...",CXIAMERI,American Station Inbound,CHABALEF,Charlotte Ave & Balmy Ave Eb,24.662,2.685420,9.183666,8.902,3.746,...,1.876,1.518,238,2.025,4/27/22-5/25/22,6am-9am,m-f,inbound,240,0.991667
562,"LINESTRING (-86.80505 36.15676, -86.80479 36.1...",CXICENTE,23rd Ave / Centennial Station Inbound,CHA20AEF,Charlotte Ave & 20th Ave Eb,24.161,1.953690,12.366855,6.845,2.752,...,1.439,1.282,229,1.514,4/27/22-5/25/22,6am-9am,m-f,inbound,240,0.954167
563,"LINESTRING (-86.85058 36.15184, -86.85031 36.1...",CXI53AVE,53rd Ave Station Inbound,CXIRICHL,46th Ave/Richland Park Station Inbound,24.270,6.419932,3.780414,154.866,17.493,...,1.578,1.315,234,1.693,4/27/22-5/25/22,6am-9am,m-f,inbound,240,0.975000
564,"LINESTRING (-86.83597 36.15238, -86.83570 36.1...",CXISYLPA,42nd Ave/Sylvan Park Station Inbound,CHA39AEF,Charlotte Ave & 39th Ave N Eb,23.766,3.739060,6.356143,9.463,5.835,...,1.935,1.480,233,2.315,4/27/22-5/25/22,6am-9am,m-f,inbound,240,0.970833


In [17]:
all_am_inschool = pd.concat([mfam_jantofeb, mfam_febtomar, mfam_martoapr, mfam_aprtomay])
all_am_inschool = all_am_inschool.reset_index()

all_am_inschool

,index,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,times,days_of_week,direction,max_seg_count,proportion_of_segments
0,0,"LINESTRING (-86.83200 36.15252, -86.83173 36.1...",CHA39AEF,Charlotte Ave & 39th Ave N Eb,CXISYLHE,37th Ave/Sylvan Heights Station Inbound,24.375,2.139055,11.395219,7.813,...,1.431,1.285,253,1.490,1/25/22-2/24/22,6am-9am,m-f,inbound,262,0.965649
1,1,"LINESTRING (-86.86416 36.14888, -86.86390 36.1...",CHABALEF,Charlotte Ave & Balmy Ave Eb,CHALELEM,Charlotte Pike & Lellyett St Eb,23.507,1.545192,15.212996,4.745,...,1.247,1.064,245,1.312,1/25/22-2/24/22,6am-9am,m-f,inbound,262,0.935115
2,2,"LINESTRING (-86.79194 36.16148, -86.79169 36.1...",CXI17AVE,17th Ave Station Inbound,CXINGULC,11th Ave / N Gulch Station Inbound,24.691,2.323759,10.625456,5.661,...,1.800,1.280,253,1.921,1/25/22-2/24/22,6am-9am,m-f,inbound,262,0.965649
3,3,"LINESTRING (-86.84193 36.15216, -86.84166 36.1...",CXIRICHL,46th Ave/Richland Park Station Inbound,CXISYLPA,42nd Ave/Sylvan Park Station Inbound,24.527,2.678840,9.155829,15.590,...,1.683,0.969,256,1.843,1/25/22-2/24/22,6am-9am,m-f,inbound,262,0.977099
4,4,"LINESTRING (-86.87277 36.14445, -86.87258 36.1...",PREWILNN,Premier Dr & Wilclay Dr Nb,CXIAMERI,American Station Inbound,24.433,4.294515,5.689350,27.961,...,2.407,1.326,241,2.770,1/25/22-2/24/22,6am-9am,m-f,inbound,262,0.919847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2262,561,"LINESTRING (-86.87143 36.14546, -86.87124 36.1...",CXIAMERI,American Station Inbound,CHABALEF,Charlotte Ave & Balmy Ave Eb,24.662,2.581885,9.551936,6.928,...,1.780,1.181,235,1.931,4/27/22-5/25/22,6am-9am,m-f,inbound,242,0.971074
2263,562,"LINESTRING (-86.80505 36.15676, -86.80479 36.1...",CXICENTE,23rd Ave / Centennial Station Inbound,CHA20AEF,Charlotte Ave & 20th Ave Eb,24.161,1.968857,12.271587,10.811,...,1.467,1.149,238,1.557,4/27/22-5/25/22,6am-9am,m-f,inbound,242,0.983471
2264,563,"LINESTRING (-86.85058 36.15184, -86.85031 36.1...",CXI53AVE,53rd Ave Station Inbound,CXIRICHL,46th Ave/Richland Park Station Inbound,24.270,6.782868,3.578132,84.840,...,1.621,1.184,242,1.708,4/27/22-5/25/22,6am-9am,m-f,inbound,242,1.000000
2265,564,"LINESTRING (-86.88379 36.14136, -86.88404 36.1...",ANNCHAWM,Annex Ave & Charlotte Pike Wb,PREWILNN,Premier Dr & Wilclay Dr Nb,24.837,3.491268,7.114034,8.314,...,2.457,1.605,239,2.659,4/27/22-5/25/22,6am-9am,m-f,inbound,242,0.987603


In [21]:
all_am_summer = pd.concat([mfam_maytojun, mfam_juntojul])
all_am_summer = all_am_summer.reset_index()

all_am_summer

,index,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,times,days_of_week,direction,max_seg_count,proportion_of_segments
0,0,"LINESTRING (-86.83200 36.15252, -86.83173 36.1...",CHA39AEF,Charlotte Ave & 39th Ave N Eb,CXISYLHE,37th Ave/Sylvan Heights Station Inbound,24.375,2.532788,9.623782,12.290,...,1.429,1.139,245,1.498,4/27/22-5/25/22,6am-9am,m-f,inbound,252,0.972222
1,1,"LINESTRING (-86.86416 36.14888, -86.86390 36.1...",CHABALEF,Charlotte Ave & Balmy Ave Eb,CHALELEM,Charlotte Pike & Lellyett St Eb,23.507,1.512034,15.546608,5.910,...,1.243,1.007,235,1.331,4/27/22-5/25/22,6am-9am,m-f,inbound,252,0.932540
2,2,"LINESTRING (-86.79194 36.16148, -86.79169 36.1...",CXI17AVE,17th Ave Station Inbound,CXINGULC,11th Ave / N Gulch Station Inbound,24.691,2.416913,10.215924,11.560,...,1.761,1.389,240,1.884,4/27/22-5/25/22,6am-9am,m-f,inbound,252,0.952381
3,3,"LINESTRING (-86.84193 36.15216, -86.84166 36.1...",CXIRICHL,46th Ave/Richland Park Station Inbound,CXISYLPA,42nd Ave/Sylvan Park Station Inbound,24.527,2.760904,8.883684,15.635,...,1.712,1.401,249,1.894,4/27/22-5/25/22,6am-9am,m-f,inbound,252,0.988095
4,4,"LINESTRING (-86.87277 36.14445, -86.87258 36.1...",PREWILNN,Premier Dr & Wilclay Dr Nb,CXIAMERI,American Station Inbound,24.433,4.439903,5.503048,10.256,...,2.709,1.747,247,3.081,4/27/22-5/25/22,6am-9am,m-f,inbound,252,0.980159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,561,"LINESTRING (-86.87143 36.14546, -86.87124 36.1...",CXIAMERI,American Station Inbound,CHABALEF,Charlotte Ave & Balmy Ave Eb,24.662,2.685420,9.183666,8.902,...,1.876,1.518,238,2.025,4/27/22-5/25/22,6am-9am,m-f,inbound,240,0.991667
1128,562,"LINESTRING (-86.80505 36.15676, -86.80479 36.1...",CXICENTE,23rd Ave / Centennial Station Inbound,CHA20AEF,Charlotte Ave & 20th Ave Eb,24.161,1.953690,12.366855,6.845,...,1.439,1.282,229,1.514,4/27/22-5/25/22,6am-9am,m-f,inbound,240,0.954167
1129,563,"LINESTRING (-86.85058 36.15184, -86.85031 36.1...",CXI53AVE,53rd Ave Station Inbound,CXIRICHL,46th Ave/Richland Park Station Inbound,24.270,6.419932,3.780414,154.866,...,1.578,1.315,234,1.693,4/27/22-5/25/22,6am-9am,m-f,inbound,240,0.975000
1130,564,"LINESTRING (-86.83597 36.15238, -86.83570 36.1...",CXISYLPA,42nd Ave/Sylvan Park Station Inbound,CHA39AEF,Charlotte Ave & 39th Ave N Eb,23.766,3.739060,6.356143,9.463,...,1.935,1.480,233,2.315,4/27/22-5/25/22,6am-9am,m-f,inbound,240,0.970833


In [28]:
am_inschool = all_am_inschool.loc[all_am_inschool['proportion_of_segments'] >= 0.5]\

am_inschool.describe()

,index,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,countForSegment,percent20Secs,max_seg_count,proportion_of_segments
count,2260.00000,2260.000000,2260.000000,2260.000000,2260.000000,2260.000000,2260.000000,2260.000000,2260.000000,2260.000000,2260.000000,2260.000000,2260.000000,2260.000000,2260.000000,2260.000000
mean,282.80177,24.452628,3.300496,10.359610,24.995695,5.882825,4.167797,3.285735,2.382921,1.942979,1.650646,1.175879,228.414159,1.802613,234.000000,0.976361
std,163.83248,0.373907,3.972234,3.901991,48.851215,10.691993,7.468676,5.250068,2.117715,0.737111,0.416083,0.293210,26.121844,0.547398,26.689233,0.022246
min,0.00000,23.427000,1.402672,0.499534,1.972000,1.609000,1.518000,1.457000,1.356500,1.280000,1.177000,0.563000,139.000000,1.243000,190.000000,0.731579
25%,141.00000,24.270000,1.809071,7.685138,5.554000,2.162750,1.920250,1.791000,1.638000,1.518000,1.374000,0.999750,190.000000,1.459000,229.000000,0.966942
50%,283.50000,24.486000,2.311541,10.608185,10.366000,3.095000,2.524000,2.262500,1.959000,1.758000,1.541500,1.178000,237.000000,1.658000,242.000000,0.979339
75%,425.00000,24.691000,3.202501,13.556485,25.887750,4.850500,3.677500,3.059500,2.452000,2.110250,1.789250,1.306000,242.000000,1.966000,247.000000,0.991736
max,566.00000,24.929000,49.017718,17.582870,908.216000,111.068000,96.403000,78.093000,41.542000,11.763000,4.785000,2.739000,262.000000,7.856000,262.000000,1.000000


In [32]:
am_inschool

,index,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,times,days_of_week,direction,max_seg_count,proportion_of_segments
0,0,"LINESTRING (-86.83200 36.15252, -86.83173 36.1...",CHA39AEF,Charlotte Ave & 39th Ave N Eb,CXISYLHE,37th Ave/Sylvan Heights Station Inbound,24.375,2.139055,11.395219,7.813,...,1.431,1.285,253,1.490,1/25/22-2/24/22,6am-9am,m-f,inbound,262,0.965649
1,1,"LINESTRING (-86.86416 36.14888, -86.86390 36.1...",CHABALEF,Charlotte Ave & Balmy Ave Eb,CHALELEM,Charlotte Pike & Lellyett St Eb,23.507,1.545192,15.212996,4.745,...,1.247,1.064,245,1.312,1/25/22-2/24/22,6am-9am,m-f,inbound,262,0.935115
2,2,"LINESTRING (-86.79194 36.16148, -86.79169 36.1...",CXI17AVE,17th Ave Station Inbound,CXINGULC,11th Ave / N Gulch Station Inbound,24.691,2.323759,10.625456,5.661,...,1.800,1.280,253,1.921,1/25/22-2/24/22,6am-9am,m-f,inbound,262,0.965649
3,3,"LINESTRING (-86.84193 36.15216, -86.84166 36.1...",CXIRICHL,46th Ave/Richland Park Station Inbound,CXISYLPA,42nd Ave/Sylvan Park Station Inbound,24.527,2.678840,9.155829,15.590,...,1.683,0.969,256,1.843,1/25/22-2/24/22,6am-9am,m-f,inbound,262,0.977099
4,4,"LINESTRING (-86.87277 36.14445, -86.87258 36.1...",PREWILNN,Premier Dr & Wilclay Dr Nb,CXIAMERI,American Station Inbound,24.433,4.294515,5.689350,27.961,...,2.407,1.326,241,2.770,1/25/22-2/24/22,6am-9am,m-f,inbound,262,0.919847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2262,561,"LINESTRING (-86.87143 36.14546, -86.87124 36.1...",CXIAMERI,American Station Inbound,CHABALEF,Charlotte Ave & Balmy Ave Eb,24.662,2.581885,9.551936,6.928,...,1.780,1.181,235,1.931,4/27/22-5/25/22,6am-9am,m-f,inbound,242,0.971074
2263,562,"LINESTRING (-86.80505 36.15676, -86.80479 36.1...",CXICENTE,23rd Ave / Centennial Station Inbound,CHA20AEF,Charlotte Ave & 20th Ave Eb,24.161,1.968857,12.271587,10.811,...,1.467,1.149,238,1.557,4/27/22-5/25/22,6am-9am,m-f,inbound,242,0.983471
2264,563,"LINESTRING (-86.85058 36.15184, -86.85031 36.1...",CXI53AVE,53rd Ave Station Inbound,CXIRICHL,46th Ave/Richland Park Station Inbound,24.270,6.782868,3.578132,84.840,...,1.621,1.184,242,1.708,4/27/22-5/25/22,6am-9am,m-f,inbound,242,1.000000
2265,564,"LINESTRING (-86.88379 36.14136, -86.88404 36.1...",ANNCHAWM,Annex Ave & Charlotte Pike Wb,PREWILNN,Premier Dr & Wilclay Dr Nb,24.837,3.491268,7.114034,8.314,...,2.457,1.605,239,2.659,4/27/22-5/25/22,6am-9am,m-f,inbound,242,0.987603


In [31]:
am_summer = all_am_summer.loc[all_am_summer['proportion_of_segments'] >= 0.5]\

am_summer.describe()

,index,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,countForSegment,percent20Secs,max_seg_count,proportion_of_segments
count,1130.000000,1130.000000,1130.000000,1130.000000,1130.000000,1130.000000,1130.000000,1130.000000,1130.000000,1130.000000,1130.000000,1130.000000,1130.000000,1130.000000,1130.000000,1130.000000
mean,282.504425,24.452628,3.287848,10.666016,24.711032,5.814158,4.222130,3.373173,2.424277,1.956472,1.618922,1.117849,239.614159,1.796961,246.000000,0.973958
std,163.606775,0.373990,4.290944,4.095163,56.539483,11.143006,8.047563,5.834034,2.576865,0.971432,0.405034,0.297518,8.170231,0.582937,6.002657,0.019339
min,0.000000,23.427000,1.389111,0.508919,2.042000,1.612000,1.520000,1.428000,1.355000,1.259000,1.160000,0.622000,188.000000,1.218000,240.000000,0.753968
25%,141.000000,24.270000,1.765278,7.771503,4.732500,2.119250,1.892000,1.751750,1.616625,1.509000,1.356000,0.965000,234.000000,1.452250,240.000000,0.966667
50%,283.000000,24.486000,2.270066,10.807093,7.953000,2.967000,2.481500,2.228000,1.946250,1.742500,1.512000,1.102000,237.000000,1.647000,246.000000,0.979167
75%,424.000000,24.691000,3.149889,13.895917,21.409500,4.626750,3.662750,3.024250,2.435375,2.117750,1.744750,1.260000,248.000000,1.957000,252.000000,0.987500
max,565.000000,24.929000,48.113787,17.754521,952.070000,108.863000,90.760000,77.546000,47.603000,17.537000,5.257000,2.662000,252.000000,8.654000,252.000000,1.000000


In [33]:
am_summer

,index,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,times,days_of_week,direction,max_seg_count,proportion_of_segments
0,0,"LINESTRING (-86.83200 36.15252, -86.83173 36.1...",CHA39AEF,Charlotte Ave & 39th Ave N Eb,CXISYLHE,37th Ave/Sylvan Heights Station Inbound,24.375,2.532788,9.623782,12.290,...,1.429,1.139,245,1.498,4/27/22-5/25/22,6am-9am,m-f,inbound,252,0.972222
1,1,"LINESTRING (-86.86416 36.14888, -86.86390 36.1...",CHABALEF,Charlotte Ave & Balmy Ave Eb,CHALELEM,Charlotte Pike & Lellyett St Eb,23.507,1.512034,15.546608,5.910,...,1.243,1.007,235,1.331,4/27/22-5/25/22,6am-9am,m-f,inbound,252,0.932540
2,2,"LINESTRING (-86.79194 36.16148, -86.79169 36.1...",CXI17AVE,17th Ave Station Inbound,CXINGULC,11th Ave / N Gulch Station Inbound,24.691,2.416913,10.215924,11.560,...,1.761,1.389,240,1.884,4/27/22-5/25/22,6am-9am,m-f,inbound,252,0.952381
3,3,"LINESTRING (-86.84193 36.15216, -86.84166 36.1...",CXIRICHL,46th Ave/Richland Park Station Inbound,CXISYLPA,42nd Ave/Sylvan Park Station Inbound,24.527,2.760904,8.883684,15.635,...,1.712,1.401,249,1.894,4/27/22-5/25/22,6am-9am,m-f,inbound,252,0.988095
4,4,"LINESTRING (-86.87277 36.14445, -86.87258 36.1...",PREWILNN,Premier Dr & Wilclay Dr Nb,CXIAMERI,American Station Inbound,24.433,4.439903,5.503048,10.256,...,2.709,1.747,247,3.081,4/27/22-5/25/22,6am-9am,m-f,inbound,252,0.980159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,561,"LINESTRING (-86.87143 36.14546, -86.87124 36.1...",CXIAMERI,American Station Inbound,CHABALEF,Charlotte Ave & Balmy Ave Eb,24.662,2.685420,9.183666,8.902,...,1.876,1.518,238,2.025,4/27/22-5/25/22,6am-9am,m-f,inbound,240,0.991667
1128,562,"LINESTRING (-86.80505 36.15676, -86.80479 36.1...",CXICENTE,23rd Ave / Centennial Station Inbound,CHA20AEF,Charlotte Ave & 20th Ave Eb,24.161,1.953690,12.366855,6.845,...,1.439,1.282,229,1.514,4/27/22-5/25/22,6am-9am,m-f,inbound,240,0.954167
1129,563,"LINESTRING (-86.85058 36.15184, -86.85031 36.1...",CXI53AVE,53rd Ave Station Inbound,CXIRICHL,46th Ave/Richland Park Station Inbound,24.270,6.419932,3.780414,154.866,...,1.578,1.315,234,1.693,4/27/22-5/25/22,6am-9am,m-f,inbound,240,0.975000
1130,564,"LINESTRING (-86.83597 36.15238, -86.83570 36.1...",CXISYLPA,42nd Ave/Sylvan Park Station Inbound,CHA39AEF,Charlotte Ave & 39th Ave N Eb,23.766,3.739060,6.356143,9.463,...,1.935,1.480,233,2.315,4/27/22-5/25/22,6am-9am,m-f,inbound,240,0.970833


# Start of M-F outbound df, sorted by 30 day chunks Jan 25-July26

In [ ]:
# URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

# params = {
#     'direction': '0',
#     'startDate': '01-25-2022',
#     'beginTime' : '06:00',
#     'daysOfWeek' : '1,2,3,4,5',
#     'endDate' : '02-24-2022',
#     'endTime' : '09:00',
#     'format' : 'geojson',
#     'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
#     'resolution': 'hiRes',
# }

# params1 = {
#     'direction': '0',
#     'startDate': '01-25-2022',
#     'beginTime' : '06:00',
#     'daysOfWeek' : '1,2,3,4,5',
#     'endDate' : '02-24-2022',
#     'endTime' : '09:00',
#     'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
#     'resolution': 'hiRes',
# }

# headers = {'Authorization': Authorization,
#           'Content-Type': 'application/json'}

# janfeboutbound = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

# janfeboutbound = janfeboutbound.loc[janfeboutbound['countForSegment'] > janfeboutbound['countForSegment'].max()/2]

# janfeboutbound

In [ ]:
# URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

# params = {
#     'direction': '0',
#     'startDate': '02-25-2022',
#     'beginTime' : '06:00',
#     'daysOfWeek' : '1,2,3,4,5',
#     'endDate' : '03-26-2022',
#     'endTime' : '09:00',
#     'format' : 'geojson',
#     'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
#     'resolution': 'hiRes',
# }

# params1 = {
#     'direction': '0',
#     'startDate': '02-25-2022',
#     'beginTime' : '06:00',
#     'daysOfWeek' : '1,2,3,4,5',
#     'endDate' : '03-26-2022',
#     'endTime' : '09:00',
#     'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
#     'resolution': 'hiRes',
# }

# headers = {'Authorization': Authorization,
#           'Content-Type': 'application/json'}

# febmaroutbound = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

# febmaroutbound = febmaroutbound.loc[febmaroutbound['countForSegment'] > febmaroutbound['countForSegment'].max()/2]

# febmaroutbound

In [ ]:
# URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

# params = {
#     'direction': '0',
#     'startDate': '03-27-2022',
#     'beginTime' : '06:00',
#     'daysOfWeek' : '1,2,3,4,5',
#     'endDate' : '04-26-2022',
#     'endTime' : '09:00',
#     'format' : 'geojson',
#     'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
#     'resolution': 'hiRes',
# }

# params1 = {
#     'direction': '0',
#     'startDate': '03-27-2022',
#     'beginTime' : '06:00',
#     'daysOfWeek' : '1,2,3,4,5',
#     'endDate' : '04-26-2022',
#     'endTime' : '09:00',
#     'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
#     'resolution': 'hiRes',
# }

# headers = {'Authorization': Authorization,
#           'Content-Type': 'application/json'}

# maraproutbound = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

# maraproutbound = maraproutbound.loc[maraproutbound['countForSegment'] > maraproutbound['countForSegment'].max()/2]

# maraproutbound

In [ ]:
# URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

# params = {
#     'direction': '0',
#     'startDate': '04-27-2022',
#     'beginTime' : '06:00',
#     'daysOfWeek' : '1,2,3,4,5',
#     'endDate' : '05-25-2022',
#     'endTime' : '09:00',
#     'format' : 'geojson',
#     'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
#     'resolution': 'hiRes',
# }

# params1 = {
#     'direction': '0',
#     'startDate': '04-27-2022',
#     'beginTime' : '06:00',
#     'daysOfWeek' : '1,2,3,4,5',
#     'endDate' : '05-25-2022',
#     'endTime' : '09:00',
#     'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
#     'resolution': 'hiRes',
# }

# headers = {'Authorization': Authorization,
#           'Content-Type': 'application/json'}

# aprmayoutbound = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

# aprmayoutbound = aprmayoutbound.loc[aprmayoutbound['countForSegment'] > aprmayoutbound['countForSegment'].max()/2]

# aprmayoutbound

In [ ]:
# URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

# params = {
#     'direction': '0',
#     'startDate': '05-26-2022',
#     'beginTime' : '06:00',
#     'daysOfWeek' : '1,2,3,4,5',
#     'endDate' : '06-25-2022',
#     'endTime' : '09:00',
#     'format' : 'geojson',
#     'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
#     'resolution': 'hiRes',
# }

# params1 = {
#     'direction': '0',
#     'startDate': '05-26-2022',
#     'beginTime' : '06:00',
#     'daysOfWeek' : '1,2,3,4,5',
#     'endDate' : '06-25-2022',
#     'endTime' : '09:00',
#     'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
#     'resolution': 'hiRes',
# }

# headers = {'Authorization': Authorization,
#           'Content-Type': 'application/json'}

# mayjunoutbound = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

# mayjunoutbound = mayjunoutbound.loc[mayjunoutbound['countForSegment'] > mayjunoutbound['countForSegment'].max()/2]

# mayjunoutbound

In [ ]:
# URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

# params = {
#     'direction': '0',
#     'startDate': '06-26-2022',
#     'beginTime' : '06:00',
#     'daysOfWeek' : '1,2,3,4,5',
#     'endDate' : '07-26-2022',
#     'endTime' : '09:00',
#     'format' : 'geojson',
#     'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
#     'resolution': 'hiRes',
# }

# params1 = {
#     'direction': '0',
#     'startDate': '06-26-2022',
#     'beginTime' : '06:00',
#     'daysOfWeek' : '1,2,3,4,5',
#     'endDate' : '07-26-2022',
#     'endTime' : '09:00',
#     'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
#     'resolution': 'hiRes',
# }

# headers = {'Authorization': Authorization,
#           'Content-Type': 'application/json'}

# junjuloutbound = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

# junjuloutbound = junjuloutbound.loc[junjuloutbound['countForSegment'] > junjuloutbound['countForSegment'].max()/2]

# junjuloutbound